# Mutant v23.08

## 1. Change Logs
- Modify TP/SL to 0.6%/1.8%

## 2. Backtest Method
    30 days as a group, randomly choose 30 continuos 30 days to see the performance

In [1]:
# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)
import datetime
# import os.path
import sys

import pandas as pd
import backtrader as bt
import backtrader.indicators as ta

import matplotlib
import tkinter
matplotlib.use('TKAgg')
# matplotlib.use('QT5Agg')

from mutant.model import Mutant
# from mutant.strategy import MutantBacktrader

raw_data_path = "../data/BTCUSD_latest.csv"
# raw_data_path = "../data/Raw_BTCUSDT1708-2303.csv"

## 3. Load data

In [2]:
dataframe = pd.read_csv(raw_data_path,
                                parse_dates=True,
                                index_col=0)
dataframe.index = pd.to_datetime(dataframe.index, format='ISO8601')

In [3]:
# start = '2022-08-01 00:00:00'
# end = '2022-09-30 00:00:00'
start = '2023-05-30 00:00:00'
end = '2023-07-01 00:00:00'
df = dataframe.loc[start:end]

In [4]:
df = df.groupby(pd.Grouper(freq='5Min')).agg({"open": "first", 
                                              "high": "max",
                                              "low": "min",
                                              "close": "last"})
df

,open,high,low,close
date,,,,
2023-05-30 00:00:00,27736.39,27745.93,27732.55,27745.92
2023-05-30 00:05:00,27745.92,27768.15,27739.97,27768.14
2023-05-30 00:10:00,27768.15,27768.15,27715.01,27729.14
2023-05-30 00:15:00,27729.15,27749.15,27729.14,27739.63
2023-05-30 00:20:00,27739.63,27739.64,27720.00,27722.47
...,...,...,...,...
2023-06-30 23:40:00,30483.97,30520.02,30483.97,30513.45
2023-06-30 23:45:00,30513.46,30513.46,30468.00,30474.99
2023-06-30 23:50:00,30474.99,30499.43,30465.54,30491.23


In [5]:
data = bt.feeds.PandasData(
    dataname=df,
    # fromdate=datetime.datetime(2022, 9, 1),
    # todate=datetime.datetime(2022, 9, 2),
    
    # timeframe=bt.TimeFrame.Minutes,
    datetime=None,
    # time=-1,
    # open=1,
    # high=2,
    # low=3,
    # close=4,
    # volume=5,
    # openinterest=-1
)

In [6]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.model = Mutant()
        self.params = self.model.params
        self.dataclose = self.datas[0].close
        self._generate_indicators()
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell(): 
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        # self.log('Close, %.2f' % self.dataclose[0])
        candle = {
            "open": self.datas[0].open[0],
            "high": self.datas[0].high[0],
            "low": self.datas[0].low[0],
            "close": self.datas[0].close[0]
        }
        indicators = {
            "ema_1": self.ema_1[0],
            "ema_2": self.ema_2[0],
            "ema_3": self.ema_3[0],
            "macd": self.macd[0],
            "macd_signal": self.macd_signal[0],
            "macd_hist": self.macd_hist[0],
            "macd_avg": self.macd_avg[0],
            "rsi": self.rsi[0]
        }

        if not self.position:
            order = self.model.get_order(candle, indicators)
            if order is not None:
                if order["trade_type"]=="Long":
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.order = self.buy_bracket(exectype=bt.Order.Market,
                                                 limitprice=order["tp_price"],
                                                 stopprice=order["sl_price"])
                if order["trade_type"]=="Short":
                    self.log('SELL CREATE, %.2f' % self.dataclose[0])
                    self.order = self.sell_bracket(exectype=bt.Order.Market,
                                                 limitprice=order["tp_price"],
                                                 stopprice=order["sl_price"])
                    
    def _generate_indicators(self):
        # EMA - Moving average exponential  
        self.ema_1 = ta.EMA(self.dataclose, period=self.params["ema_1_length"])
        self.ema_2 = ta.EMA(self.dataclose, period=self.params["ema_2_length"])
        self.ema_3 = ta.EMA(self.dataclose, period=self.params["ema_3_length"])
        # MACD - Moving average convergence divergence
        macd = ta.MACD(
            period_me1=self.params["macd_fast_length"],
            period_me2=self.params["macd_slow_length"],
            period_signal=self.params["macd_signal_length"])
        self.macd = macd.macd
        self.macd_signal = macd.signal
        self.macd_hist = ta.MACDHisto(
            period_me1=self.params["macd_fast_length"],
            period_me2=self.params["macd_slow_length"],
            period_signal=self.params["macd_signal_length"])
        self.macd_avg = ta.SMA(self.macd_hist, period=self.params["macd_average_length"])
        self.rsi = ta.RSI(self.dataclose, period=self.params["rsi_length"])
        return None

In [7]:
cerebro = bt.Cerebro()
# cerebro.addstrategy(MutantBacktrader)
cerebro.addstrategy(TestStrategy)
cerebro.adddata(data)

In [8]:
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.0004)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2023-05-30T20:55:00, SELL CREATE, 27762.51
2023-05-30T21:00:00, SELL EXECUTED, Price: 27762.52, Cost: -27762.52, Comm 11.11
2023-05-31T02:05:00, BUY EXECUTED, Price: 27585.00, Cost: -27762.52, Comm 11.03
2023-05-31T02:05:00, Order Canceled/Margin/Rejected
2023-05-31T02:05:00, OPERATION PROFIT, GROSS 177.52, NET 155.38
2023-06-01T21:05:00, BUY CREATE, 26917.18
2023-06-01T21:10:00, BUY EXECUTED, Price: 26917.18, Cost: 26917.18, Comm 10.77
2023-06-02T04:40:00, SELL EXECUTED, Price: 27104.50, Cost: 26917.18, Comm 10.84
2023-06-02T04:40:00, Order Canceled/Margin/Rejected
2023-06-02T04:40:00, OPERATION PROFIT, GROSS 187.32, NET 165.71
2023-06-02T18:15:00, SELL CREATE, 27039.03
2023-06-02T18:20:00, SELL EXECUTED, Price: 27039.04, Cost: -27039.04, Comm 10.82
2023-06-05T02:55:00, BUY EXECUTED, Price: 26871.39, Cost: -27039.04, Comm 10.75
2023-06-05T02:55:00, Order Canceled/Margin/Rejected
2023-06-05T02:55:00, OPERATION PROFIT, GROSS 167.65, NET 146.09
2023-06